#### Pre-Processing

In [1]:
# Parse a PDF as well as a email invoice
from src.utils import preprocess_aor, preprocess_invoice

# preprocess_aor(aor_dir="database/aor")

preprocess_invoice(invoice_dir="database/invoice")

Preprocessing Invoices...


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [3]:
!pip install poppler

ERROR: Could not find a version that satisfies the requirement poppler (from versions: none)
ERROR: No matching distribution found for poppler


#### Search & Query

In [ ]:
# Chat with the preprocessed AOR for real-time chat based on RAG 
from src.aor import load_aors
from src.search import search_aor_with_item, search_aor_with_no


aor_list = load_aors()

# Quick routing: Item-based & Number-baseds
no_query = "DH928"
matching_aors = search_aor_with_no(no_query, aor_list)


print("Item-based AOR searching:")
for aor in matching_aors[:1]:
    print(f"AOR No: {aor.no}")
    print(f"Description: {aor.description}")
    print(f"Items: {aor.items}")
    print(f"Budgets: {aor.budgets}")
    print(f"Expiry Date: {aor.expiry_date}")
    print("---")

#### RAG-based chat 

In [ ]:
from src.agent import RagAgent 

# Example usage
agent = RagAgent()
user_message = "Tell me about AOR DH928"
agent_response = agent.chat(user_message)

user_message = "What is the invoice no.01141833 about?"
agent_response = agent.chat(user_message)

user_message = "Is this invoice covered by the AOR?"
agent_response = agent.chat(user_message)

In [4]:
from src.agent import RagAgent 

agent = RagAgent()
user_message = "Tell me about AOR CAMP 1 OPS COST REQUIREMENTS"
agent_response = agent.chat(user_message)

user_message = "What is the invoice no.1234567 about?"
agent_response = agent.chat(user_message)

# user_message = "Is this invoice covered by the AOR?"
# agent_response = agent.chat(user_message)

Calling Function:  search_aor_with_item  | Query:  CAMP 1 OPS COST REQUIREMENTS
Line 90: Item matching aors:  [AOR(items=['Telco Subscription', 'Bank charges (Cheque Book and annual bank statement)', 'Newspaper and magazine Subscription', 'HSP', 'Antigen Test Kit and PCR Test', 'Annual Airport pass', 'Purchase of Stationaries, office equipment and birthday cards', 'Ad-hoc Service and Maintenance', 'Printing of Name Card', 'Dip Bag via MFA'], budgets=[424361.0, 1103.0, 947.0, 981.0, 611.0, 1184.0, 194.0, 1343.0, 699.0, 1158.0], no='CAMP 1 OPS COST REQUIREMENTS', description='Approval of requirement for Operations Cost requirements at an estimated cost.', expiry_date='Not specified', pdf_text="Message Classification: Restricted\r\n\r\nApproved. Tks.\r\n\r\nBest,\r\nHDM\r\n\r\nFrom: Seow Yuxin <yuxin.seow@temus.com>\r\nDate: Thursday, 15 August 2024 at 5:01\u202fPM\r\nTo: Yu Fangyuan <Fangyuan.Yu@temus.com>\r\nSubject: [SAMPLE AOR] [For HDM's approval] APPROVAL OF REQUIREMENT FOR CAMP 1 O

In [6]:
agent.memory.aor_list

[AOR(items=['Telco Subscription', 'Bank charges (Cheque Book and annual bank statement)', 'Newspaper and magazine Subscription', 'HSP', 'Antigen Test Kit and PCR Test', 'Annual Airport pass', 'Purchase of Stationaries, office equipment and birthday cards', 'Ad-hoc Service and Maintenance', 'Printing of Name Card', 'Dip Bag via MFA'], budgets=[424361.0, 1103.0, 947.0, 981.0, 611.0, 1184.0, 194.0, 1343.0, 699.0, 1158.0], no='CAMP 1 OPS COST REQUIREMENTS', description='Approval of requirement for Operations Cost requirements at an estimated cost.', expiry_date='Not specified', pdf_text="Message Classification: Restricted\r\n\r\nApproved. Tks.\r\n\r\nBest,\r\nHDM\r\n\r\nFrom: Seow Yuxin <yuxin.seow@temus.com>\r\nDate: Thursday, 15 August 2024 at 5:01\u202fPM\r\nTo: Yu Fangyuan <Fangyuan.Yu@temus.com>\r\nSubject: [SAMPLE AOR] [For HDM's approval] APPROVAL OF REQUIREMENT FOR CAMP 1 OPS COST REQUIREMENTS [Restricted]\r\nMessage Classification: Restricted\r\n\r\nHDM sir,\r\n\r\nAPPROVAL OF REQ

In [7]:
agent.memory.invoice_list

[Invoice(no='1234567', date='2024-08-15', currency='Thai Baht', seller='Temus', items=['Telecom service fee', 'Bank statement and annual report fee', 'Membership fee for journals and publications', 'Service fee', 'Antigen test kit and PCR test', 'Annual pass card', 'Purchase of cooling equipment, office equipment and ERP', 'Filing and archiving service', 'Card printing service', 'Express document service'], amounts=[424361.0, 1103.0, 947.0, 981.0, 611.0, 1184.0, 194.0, 1343.0, 699.0, 1158.0], description='', invoice_text='Invoice\nCompany Name: Temus\nAddress: Pasir Panjang\nInvoice Number: 1234567\nInvoice Date: 15 August 2024\n\nItems\nTelecom service fee: 424,361.00 THB\nBank statement and annual report fee: 1,103.00 THB\nMembership fee for journals and publications: 947.00 THB\nService fee: 981.00 THB\nAntigen test kit and PCR test: 611.00 THB\nAnnual pass card: 1,184.00 THB\nPurchase of cooling equipment, office equipment and ERP: 194.00 THB\nFiling and archiving service: 1,343.00

In [15]:
from src.utils import *
import glob 

eml_files = glob.glob("database/aor/*.eml")
eml_file_path = eml_files[0]
email_data = read_eml(eml_file_path)

email_txt = email_data.get('body')
prompt = PARSE_AOR_PROMPT.format(pdf_txt=email_txt)
response = get_oai_response(prompt)
parsed_aor_dict = parse_json_response(response)
aor = AOR(**parsed_aor_dict)


# print(email_data)

In [13]:
# type(email_data)
email_data['body']

"Message Classification: Restricted\r\n\r\nApproved. Tks.\r\n\r\nBest,\r\nHDM\r\n\r\nFrom: Seow Yuxin <yuxin.seow@temus.com>\r\nDate: Thursday, 15 August 2024 at 5:01\u202fPM\r\nTo: Yu Fangyuan <Fangyuan.Yu@temus.com>\r\nSubject: [SAMPLE AOR] [For HDM's approval] APPROVAL OF REQUIREMENT FOR CAMP 1 OPS COST REQUIREMENTS [Restricted]\r\nMessage Classification: Restricted\r\n\r\nHDM sir,\r\n\r\nAPPROVAL OF REQUIREMENT FOR  OPS COST REQUIREMENTS FOR CAMP 1\r\n\r\nAIM:\r\n1. To seek HDM’s approval for Ops Cost requirements at an estimated cost of S$16,290 (26.56THB to 1 SGD)  – MAS Ex Rate for 2024.\r\n\r\nCOST BREAKDOWN AND JUSTIFICATION\r\n2. Approval is sought for the following requirements:\r\n\r\nItems\r\nForecast for 2022\r\n2022 Actual Expenditure (THB)\r\nForecast for FY23/24 (THB)\r\nRemarks\r\nTelco Subscription\r\n               339,488.00\r\n                                               282,907.00\r\n                                       424,361.00\r\nProjected amount derived 

In [ ]:
from PIL import Image
import glob
# glob.glob("assets/*")[1]
icon_path = "assets/flow-agent.png"
icon = Image.open(icon_path)

#### Invoice Processor

In [ ]:
# this is the last query to solve, before the front-end deployment 
# query = "Is the invoice no.01141833 covered by any of the AORs?"
# query = "What is the invoice no.01141883 about?"

# use_message = "Is the invoice no.01141833 covered by any of the AORs?"
agent_response = agent.chat("Is the invoice no.01141833 covered by any of the AORs?")

In [ ]:
# Chainlit front-end interface with logo change now
